In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#/kaggle/input/jjhgfff/civil_code_input.txt

In [ ]:
#/kaggle/input/jjjjjjjjj/civilcode-english.pdf

In [8]:
!pip install pymupdf


In [9]:
from enum import STRICT
import fitz
import re
import json

PDF_PATH = "/kaggle/input/jjjjjjjjj/civilcode-english.pdf"
OUTPUT = "_clean.json"

LAW = "National Civil Code 2017"

doc = fitz.open(PDF_PATH)

# Extract + pre-clean text
raw = []
for page in doc:
    t = page.get_text()
    t = re.sub(r'(\w)-\n(\w)', r'\1\2', t)   # fix hyphens
    t = re.sub(r'\n+', '\n', t)
    raw.append(t)

lines = [l.strip() for l in "\n".join(raw).split("\n")]

# remove pure page numbers
lines = [l for l in lines if not re.fullmatch(r'\d+', l)]

#2.Regrex(STRICT ORDER)
part_re = re.compile(r'^Part\s*[-–]?\s*(\d+)\s*(.*)', re.I)
chapter_re = re.compile(r'^Chapter\s*[-–]?\s*(\d+)', re.I)
section_re = re.compile(r'^(\d+)\.\s*(.+)')
subsection_re = re.compile(r'^\((\d+)\)')
clause_re = re.compile(r'^\(([a-z])\)')

#3.State
records = []

part = chapter = chapter_title = None
section = section_title = None
subsection = None
buffer = ""

def flush():
    global buffer
    if section and buffer.strip():
        records.append({
            "law": LAW,
            "part": part,
            "chapter": chapter,
            "chapter_title": chapter_title,
            "section": section,
            "section_title": section_title,
            "subsection": subsection,
            "text": buffer.strip(),
            "source": "penal-english.pdf"  #here replacewith ----"source": "National Penal (Code) Act, 2017 (English translation) — https://bwcimplementation.org/sites/default/files/resource/NP_National%20Penal%20Code%20Act_EN.pdf"

  
        })
    buffer = ""

i = 0
while i < len(lines):
    line = lines[i]
    i += 1
    if not line:
        continue

    # PART
    m = part_re.match(line)
    if m:
        flush()
        part = f"Part-{m.group(1)}"
        section = subsection = None
        continue

    # CHAPTER
    m = chapter_re.match(line)
    if m:
        flush()
        chapter = f"Chapter-{m.group(1)}"
        section = subsection = None

        # next non-empty = chapter title
        while i < len(lines) and not lines[i]:
            i += 1
        chapter_title = lines[i] if i < len(lines) else None
        i += 1
        continue

    # SECTION (HARD BOUNDARY)
    m = section_re.match(line)
    if m:
        flush()
        section = int(m.group(1))
        section_title = m.group(2).rstrip(":")
        subsection = None
        continue

    # SUBSECTION
    m = subsection_re.match(line)
    if m:
        flush()
        subsection = f"({m.group(1)})"
        continue

    # CLAUSE → keep inside subsection
    if clause_re.match(line):
        buffer += " " + line
        continue

    # CONTENT
    buffer += " " + line

flush()

#4. Output
with open(OUTPUT, "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"Clean legal dataset generated: {len(records)} records")

Clean legal dataset generated: 1465 records


In [12]:
from enum import Enum
import fitz
import re
import json

PDF_PATH = "/kaggle/input/jjjjjjjjj/civilcode-english.pdf"
OUTPUT = "jjjjjjjcivilcode_clean.json"
LAW = "National Civil Code 2017"

doc = fitz.open(PDF_PATH)

# Extract + pre-clean text
raw = []
for page in doc:
    t = page.get_text()
    t = re.sub(r'(\w)-\n(\w)', r'\1\2', t)   # fix hyphens
    t = re.sub(r'\n+', '\n', t)
    raw.append(t)

lines = [l.strip() for l in "\n".join(raw).split("\n")]
# remove pure page numbers and empty lines
lines = [l for l in lines if l and not re.fullmatch(r'\d+', l)]

# 2. Regex (STRICT ORDER) - MORE STRICT PATTERNS
part_re = re.compile(r'^Part\s*[-–]?\s*(\d+)\s*$', re.I)  # Part must be on its own line
chapter_re = re.compile(r'^Chapter\s*[-–]?\s*(\d+)\s*$', re.I)  # Chapter must be on its own line
# CRITICAL FIX: Section must be followed by a period and at least some title text
section_re = re.compile(r'^(\d+)\.\s+([A-Z].{3,})', re.UNICODE)  # Must have substantial title
subsection_re = re.compile(r'^\((\d+)\)\s*(.*)$')
clause_re = re.compile(r'^\(([a-z])\)\s*(.*)$')

# 3. State
records = []
part = chapter = chapter_title = None
section = section_title = None
subsection = None
buffer = ""
in_legal_content = False  # NEW: Track if we're in actual legal content

def flush():
    global buffer
    if section and buffer.strip() and in_legal_content:
        records.append({
            "law": LAW,
            "part": part,
            "chapter": chapter,
            "chapter_title": chapter_title,
            "section": section,
            "section_title": section_title,
            "subsection": subsection,
            "text": buffer.strip(),
            "source": "civilcode-english.pdf"
        })
    buffer = ""

i = 0
while i < len(lines):
    line = lines[i]
    i += 1
    
    if not line:
        continue
    
    # PART - must be standalone line
    m = part_re.match(line)
    if m:
        flush()
        part = f"Part-{m.group(1)}"
        section = subsection = None
        in_legal_content = True  # We're now in legal content
        # Next non-empty line is part title
        while i < len(lines) and not lines[i]:
            i += 1
        if i < len(lines):
            i += 1  # Skip part title
        continue
    
    # CHAPTER - must be standalone line
    m = chapter_re.match(line)
    if m:
        flush()
        chapter = f"Chapter-{m.group(1)}"
        section = subsection = None
        # next non-empty = chapter title
        while i < len(lines) and not lines[i]:
            i += 1
        chapter_title = lines[i] if i < len(lines) else None
        i += 1
        continue
    
    # SECTION (HARD BOUNDARY) - STRICTER MATCHING
    m = section_re.match(line)
    if m and in_legal_content:  # Only match sections if we're in legal content
        flush()
        section = int(m.group(1))
        section_title = m.group(2).rstrip(":")
        subsection = None
        
        # Check if title continues on next line(s)
        # If next line doesn't start with subsection/clause marker, it's part of title
        while i < len(lines):
            next_line = lines[i]
            if not next_line:
                i += 1
                continue
            # Stop if we hit a subsection, clause, or new section
            if (subsection_re.match(next_line) or 
                clause_re.match(next_line) or 
                section_re.match(next_line)):
                break
            # If line starts with lowercase or looks like continuation, add to title
            if next_line[0].islower() or section_title.endswith(','):
                section_title += " " + next_line
                i += 1
            else:
                break
        continue
    
    # SUBSECTION
    m = subsection_re.match(line)
    if m and section:  # Only match if we have a section
        flush()
        subsection = f"({m.group(1)})"
        # Add any content after the subsection marker
        if m.group(2):
            buffer = m.group(2)
        continue
    
    # CLAUSE → keep inside subsection/section
    m = clause_re.match(line)
    if m and section:  # Only match if we have a section
        buffer += " " + line
        continue
    
    # CONTENT - only add if we're in a valid section
    if section:
        buffer += " " + line

flush()

# 4. Output
with open(OUTPUT, "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"Clean legal dataset generated: {len(records)} records")

# 5. Validation - print first few sections to verify
print("\nFirst 10 sections found:")
for i, rec in enumerate(records[:10]):
    print(f"{i+1}. Section {rec['section']}: {rec['section_title'][:50]}...")

Clean legal dataset generated: 1074 records

First 10 sections found:
1. Section 10: Not to enjoy benefit of wrong: No person shall be ...
2. Section 100: Order to pay lump sum amount or alimony: Notwithst...
3. Section 101: Order to provide maintenance costs: If the wife, w...
4. Section 101: Order to provide maintenance costs: If the wife, w...
5. Section 101: Order to provide maintenance costs: If the wife, w...
6. Section 102: To be according to agreement: Notwithstanding anyt...
7. Section 105: Maternity or paternity to be determined from mothe...
8. Section 105: Maternity or paternity to be determined from mothe...
9. Section 106: Child presumed to be born from married husband: Un...
10. Section 107: Paternity may be denied:  (1) Except as otherwise ...


In [13]:
import fitz
import re
import json

PDF_PATH = "/kaggle/input/jjjjjjjjj/civilcode-english.pdf"
OUTPUT = "dkjldflcivilcode_clean.json"
LAW = "National Civil Code 2017"

doc = fitz.open(PDF_PATH)

# Extract + pre-clean text
raw = []
for page in doc:
    t = page.get_text()
    t = re.sub(r'(\w)-\n(\w)', r'\1\2', t)   # fix hyphens
    t = re.sub(r'\n+', '\n', t)
    raw.append(t)

full_text = "\n".join(raw)
lines = [l.strip() for l in full_text.split("\n") if l.strip()]

# Remove standalone page numbers
lines = [l for l in lines if not (re.fullmatch(r'\d+', l) and len(l) <= 3)]

# Patterns
part_re = re.compile(r'^Part[-–\s]*(\d+)', re.I)
chapter_re = re.compile(r'^Chapter[-–\s]*(\d+)', re.I)
# Section: number followed by dot and text (title usually starts with capital)
section_re = re.compile(r'^(\d+)\.\s+(.+)')
subsection_re = re.compile(r'^\((\d+)\)\s*(.*)$')
clause_re = re.compile(r'^\(([a-z])\)\s*(.*)$')

# State variables
records = []
current_part = None
current_chapter = None
current_chapter_title = None
current_section = None
current_section_title = None
current_subsection = None
text_buffer = []

def save_record():
    """Save current buffer as a record"""
    global text_buffer
    
    if current_section is not None and text_buffer:
        text = " ".join(text_buffer).strip()
        if text:  # Only save if there's actual text
            records.append({
                "law": LAW,
                "part": current_part,
                "chapter": current_chapter,
                "chapter_title": current_chapter_title,
                "section": current_section,
                "section_title": current_section_title,
                "subsection": current_subsection,
                "text": text,
                "source": "civilcode-english.pdf"
            })
    text_buffer = []

i = 0
while i < len(lines):
    line = lines[i]
    
    # Check for PART
    part_match = part_re.match(line)
    if part_match:
        save_record()
        current_part = f"Part-{part_match.group(1)}"
        current_section = None
        current_subsection = None
        i += 1
        # Next line might be part title - skip it
        if i < len(lines) and not section_re.match(lines[i]) and not chapter_re.match(lines[i]):
            i += 1
        continue
    
    # Check for CHAPTER
    chapter_match = chapter_re.match(line)
    if chapter_match:
        save_record()
        current_chapter = f"Chapter-{chapter_match.group(1)}"
        current_section = None
        current_subsection = None
        i += 1
        # Next line is chapter title
        if i < len(lines):
            current_chapter_title = lines[i]
            i += 1
        continue
    
    # Check for SECTION (most important - must be precise)
    section_match = section_re.match(line)
    if section_match:
        save_record()  # Save previous section
        current_section = int(section_match.group(1))
        current_section_title = section_match.group(2).rstrip(':').strip()
        current_subsection = None
        text_buffer = []
        i += 1
        continue
    
    # Check for SUBSECTION
    subsection_match = subsection_re.match(line)
    if subsection_match and current_section is not None:
        save_record()  # Save previous subsection
        current_subsection = f"({subsection_match.group(1)})"
        # Add any text after the subsection number
        rest = subsection_match.group(2).strip()
        if rest:
            text_buffer.append(rest)
        i += 1
        continue
    
    # Check for CLAUSE
    clause_match = clause_re.match(line)
    if clause_match and current_section is not None:
        # Clauses are part of the current subsection/section text
        text_buffer.append(line)
        i += 1
        continue
    
    # Regular content line
    if current_section is not None:
        text_buffer.append(line)
    
    i += 1

# Save the last record
save_record()

# Sort by section number to ensure correct order
records.sort(key=lambda x: x['section'] if x['section'] else 0)

# Output
with open(OUTPUT, "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"✓ Generated {len(records)} records")
print(f"✓ Section range: {min(r['section'] for r in records)} to {max(r['section'] for r in records)}")

# Show first 15 sections
print("\n📋 First 15 sections:")
for rec in records[:15]:
    print(f"  Section {rec['section']}: {rec['section_title'][:60]}{'...' if len(rec['section_title']) > 60 else ''}")

# Show section count by part
from collections import Counter
part_counts = Counter(r['part'] for r in records)
print("\n📊 Sections per part:")
for part in sorted(part_counts.keys()):
    print(f"  {part}: {part_counts[part]} sections")

# Check for any gaps in section numbers
sections = sorted([r['section'] for r in records])
gaps = []
for i in range(len(sections) - 1):
    if sections[i+1] - sections[i] > 1:
        gaps.append((sections[i], sections[i+1]))

if gaps:
    print("\n⚠️ Warning: Gaps in section numbers detected:")
    for start, end in gaps[:10]:  # Show first 10 gaps
        print(f"  Gap between Section {start} and {end}")
else:
    print("\n✓ No gaps in section numbering")

✓ Generated 1474 records
✓ Section range: 10 to 721

📋 First 15 sections:
  Section 10: Not to enjoy benefit of wrong: No person shall be entitled t...
  Section 100: Order to pay lump sum amount or alimony: Notwithstanding
  Section 101: Order to provide maintenance costs: If the wife, who has not
  Section 101: Order to provide maintenance costs: If the wife, who has not
  Section 101: Order to provide maintenance costs: If the wife, who has not
  Section 102: To be according to agreement: Notwithstanding anything
  Section 103: Previous child or husband to obtain property: On the death o...
  Section 104: Statute of limitation: A person aggrieved from any act done ...
  Section 105: Maternity or paternity to be determined from mother or fathe...
  Section 105: Maternity or paternity to be determined from mother or fathe...
  Section 106: Child presumed to be born from married husband: Unless
  Section 107: Paternity may be denied:  (1) Except as otherwise agreed upo...
  Section 107

In [14]:
import fitz
import re
import json

PDF_PATH = "/kaggle/input/jjjjjjjjj/civilcode-english.pdf"
OUTPUT = "aaaaaaacivilcode_clean.json"
LAW = "National Civil Code 2017"

doc = fitz.open(PDF_PATH)

# Extract text from all pages
all_text = []
for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()
    # Fix hyphenation
    text = re.sub(r'(\w)-\n(\w)', r'\1\2', text)
    all_text.append(text)

full_text = "\n".join(all_text)

# Split into lines and clean
lines = []
for line in full_text.split('\n'):
    line = line.strip()
    if line and not re.fullmatch(r'\d{1,2}', line):  # Skip page numbers
        lines.append(line)

print(f"Total lines extracted: {len(lines)}")

# More flexible patterns
part_re = re.compile(r'^Part[-–\s]*(\d+)', re.I)
chapter_re = re.compile(r'^Chapter[-–\s]*(\d+)', re.I)
# FLEXIBLE section pattern - catches variations like "1." or "1 ." or "1.Short title"
section_re = re.compile(r'^(\d+)\s*\.?\s*(.+?)(?:\s*:)?\s*(.*)$')
subsection_re = re.compile(r'^\((\d+)\)\s*(.*)$')
clause_re = re.compile(r'^\(([a-z])\)\s*(.*)$')

# State
records = []
current_part = None
current_chapter = None
current_chapter_title = None
current_section = None
current_section_title = None
current_subsection = None
buffer = []

# Track where we started finding sections
first_section_found = False

def flush():
    global buffer
    if current_section is not None and buffer:
        text = " ".join(buffer).strip()
        if text:
            records.append({
                "law": LAW,
                "part": current_part,
                "chapter": current_chapter,
                "chapter_title": current_chapter_title,
                "section": current_section,
                "section_title": current_section_title,
                "subsection": current_subsection,
                "text": text,
                "source": "civilcode-english.pdf"
            })
    buffer = []

# First pass: find where actual legal content starts (look for "1. Short title")
start_idx = 0
for i, line in enumerate(lines):
    if re.match(r'^1\s*\.?\s*Short\s+title', line, re.I):
        start_idx = max(0, i - 10)  # Start a bit before to catch Part/Chapter
        print(f"Found Section 1 at line {i}: {line}")
        break

if start_idx == 0:
    print("Warning: Could not find 'Section 1. Short title', starting from beginning")

# Second pass: parse from start_idx
i = start_idx
while i < len(lines):
    line = lines[i]
    
    # PART
    if part_re.match(line):
        flush()
        m = part_re.match(line)
        current_part = f"Part-{m.group(1)}"
        current_section = None
        current_subsection = None
        print(f"Found {current_part}")
        i += 1
        # Skip part title if next line is not a section/chapter
        if i < len(lines) and not section_re.match(lines[i]) and not chapter_re.match(lines[i]):
            i += 1
        continue
    
    # CHAPTER
    if chapter_re.match(line):
        flush()
        m = chapter_re.match(line)
        current_chapter = f"Chapter-{m.group(1)}"
        current_section = None
        current_subsection = None
        i += 1
        # Next line is chapter title
        if i < len(lines) and not section_re.match(lines[i]):
            current_chapter_title = lines[i]
            print(f"Found {current_chapter}: {current_chapter_title}")
            i += 1
        continue
    
    # SECTION - more flexible matching
    section_match = section_re.match(line)
    if section_match:
        section_num = section_match.group(1)
        # Must be a reasonable section number (1-999)
        if section_num.isdigit() and 1 <= int(section_num) <= 999:
            flush()
            current_section = int(section_num)
            # Section title is group 2 + group 3
            title_part1 = section_match.group(2).strip()
            title_part2 = section_match.group(3).strip()
            current_section_title = f"{title_part1} {title_part2}".strip().rstrip(':')
            current_subsection = None
            
            if current_section <= 20:  # Debug first 20 sections
                print(f"Section {current_section}: {current_section_title[:50]}")
            
            i += 1
            continue
    
    # SUBSECTION
    if subsection_re.match(line) and current_section is not None:
        flush()
        m = subsection_re.match(line)
        current_subsection = f"({m.group(1)})"
        rest = m.group(2).strip()
        if rest:
            buffer.append(rest)
        i += 1
        continue
    
    # CLAUSE or regular content
    if current_section is not None:
        buffer.append(line)
    
    i += 1

# Final flush
flush()

# Sort by section number
records.sort(key=lambda x: (x['section']))

# Save
with open(OUTPUT, "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"\n{'='*60}")
print(f"✓ Total records: {len(records)}")
if records:
    sections = [r['section'] for r in records]
    print(f"✓ Section range: {min(sections)} to {max(sections)}")
    
    print(f"\n📋 First 20 sections:")
    for rec in records[:20]:
        print(f"  {rec['section']:3d}. {rec['section_title'][:65]}")
    
    # Check for missing sections 1-9
    missing = [i for i in range(1, 10) if i not in sections]
    if missing:
        print(f"\n⚠️ Missing sections: {missing}")
    else:
        print(f"\n✓ Sections 1-9 all found!")
    
    # Find gaps
    gaps = []
    sorted_sections = sorted(set(sections))
    for i in range(len(sorted_sections) - 1):
        if sorted_sections[i+1] - sorted_sections[i] > 1:
            gaps.append((sorted_sections[i], sorted_sections[i+1]))
    
    if gaps:
        print(f"\n⚠️ Gaps found: {len(gaps)}")
        for start, end in gaps[:5]:
            print(f"  Gap: {start} → {end}")

Total lines extracted: 11691
Section 16: O ctober 2017
Found Part-1
Found Chapter-1: General Provisions
Section 1: .
Section 2: .
Section 3: .
Found Chapter-2: General Principles of Civil Law
Section 4: .
Section 5: .
Section 6: .
Section 7: .
Section 8: .
Section 9: .
Section 10: N ot to enjoy benefit of wrong: No person shall be
Section 11: .
Section 12: .
Section 13: .
Section 14: .
Section 15: .
Section 16: .
Found Chapter-3: Provisions Relating To Civil Rights
Section 17: .
Section 18: .
Section 19: .
Section 20: .
Found Part-2
Found Chapter-1: Provisions Relating to Natural Persons
Found Chapter-2: Provisions Relating to Legal Persons
Found Chapter-3: Provisions Relating to Bankruptcy of Natural Persons
Found Part-3
Found Chapter-1: Provisions Relating to Marriage
Found Chapter-2: Provisions Relating to Consequences of Marriage
Found Chapter-3: Provisions Relating to Divorce
Found Chapter-4: Provisions Relating to Relationship of Mother, Father and Son and
Found Chapter-5: Provis

In [15]:
import fitz
import re
import json

PDF_PATH = "/kaggle/input/jjjjjjjjj/civilcode-english.pdf"
OUTPUT = "bbbbbbbbbcivilcode_clean.json"
LAW = "National Civil Code 2017"

doc = fitz.open(PDF_PATH)

# Extract text from all pages
all_text = []
for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()
    # Fix hyphenation
    text = re.sub(r'(\w)-\n(\w)', r'\1\2', text)
    all_text.append(text)

full_text = "\n".join(all_text)

# Split into lines and clean
lines = []
for line in full_text.split('\n'):
    line = line.strip()
    if line and not re.fullmatch(r'\d{1,2}', line):  # Skip standalone page numbers
        lines.append(line)

print(f"Total lines extracted: {len(lines)}")

# Patterns
part_re = re.compile(r'^Part[-–\s]*(\d+)', re.I)
chapter_re = re.compile(r'^Chapter[-–\s]*(\d+)', re.I)
# Match section number alone OR with title
section_num_only_re = re.compile(r'^(\d+)\.\s*$')  # Just "1." or "2."
section_with_title_re = re.compile(r'^(\d+)\.\s+(.+)$')  # "100. Title text"
subsection_re = re.compile(r'^\((\d+)\)\s*(.*)$')
clause_re = re.compile(r'^\(([a-z])\)\s*(.*)$')

# State
records = []
current_part = None
current_chapter = None
current_chapter_title = None
current_section = None
current_section_title = None
current_subsection = None
buffer = []

def flush():
    global buffer
    if current_section is not None and buffer:
        text = " ".join(buffer).strip()
        if text:
            records.append({
                "law": LAW,
                "part": current_part,
                "chapter": current_chapter,
                "chapter_title": current_chapter_title,
                "section": current_section,
                "section_title": current_section_title,
                "subsection": current_subsection,
                "text": text,
                "source": "civilcode-english.pdf"
            })
    buffer = []

i = 0
while i < len(lines):
    line = lines[i]
    
    # PART
    if part_re.match(line):
        flush()
        m = part_re.match(line)
        current_part = f"Part-{m.group(1)}"
        current_section = None
        current_subsection = None
        i += 1
        # Skip part title if exists
        if i < len(lines) and not section_num_only_re.match(lines[i]) and not section_with_title_re.match(lines[i]) and not chapter_re.match(lines[i]):
            i += 1
        continue
    
    # CHAPTER
    if chapter_re.match(line):
        flush()
        m = chapter_re.match(line)
        current_chapter = f"Chapter-{m.group(1)}"
        current_section = None
        current_subsection = None
        i += 1
        # Next line is chapter title
        if i < len(lines) and not section_num_only_re.match(lines[i]) and not section_with_title_re.match(lines[i]):
            current_chapter_title = lines[i]
            i += 1
        continue
    
    # SECTION WITH TITLE (like "100. Order to pay lump sum...")
    section_match = section_with_title_re.match(line)
    if section_match:
        section_num = int(section_match.group(1))
        if 1 <= section_num <= 999:
            flush()
            current_section = section_num
            current_section_title = section_match.group(2).strip().rstrip(':')
            current_subsection = None
            i += 1
            continue
    
    # SECTION NUMBER ONLY (like "1." or "2.")
    section_num_match = section_num_only_re.match(line)
    if section_num_match:
        section_num = int(section_num_match.group(1))
        if 1 <= section_num <= 999:
            flush()
            current_section = section_num
            current_subsection = None
            i += 1
            
            # CRITICAL: Next non-empty line is the section title
            while i < len(lines):
                next_line = lines[i].strip()
                
                # Skip empty lines
                if not next_line:
                    i += 1
                    continue
                
                # If next line is a subsection or clause, no title exists
                if subsection_re.match(next_line) or clause_re.match(next_line):
                    current_section_title = ""
                    break
                
                # If next line is another section/chapter/part, no title
                if (section_num_only_re.match(next_line) or 
                    section_with_title_re.match(next_line) or
                    chapter_re.match(next_line) or 
                    part_re.match(next_line)):
                    current_section_title = ""
                    break
                
                # Otherwise, this line is the section title
                current_section_title = next_line.rstrip(':').strip()
                i += 1
                break
            
            continue
    
    # SUBSECTION
    if subsection_re.match(line) and current_section is not None:
        flush()
        m = subsection_re.match(line)
        current_subsection = f"({m.group(1)})"
        rest = m.group(2).strip()
        if rest:
            buffer.append(rest)
        i += 1
        continue
    
    # CLAUSE or regular content
    if current_section is not None:
        buffer.append(line)
    
    i += 1

# Final flush
flush()

# Sort by section number
records.sort(key=lambda x: x['section'])

# Save
with open(OUTPUT, "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"\n{'='*60}")
print(f"✓ Total records: {len(records)}")

if records:
    sections = [r['section'] for r in records]
    print(f"✓ Section range: {min(sections)} to {max(sections)}")
    
    print(f"\n📋 First 25 sections with titles:")
    for rec in records[:25]:
        title = rec['section_title'] if rec['section_title'] else "[NO TITLE]"
        print(f"  {rec['section']:3d}. {title[:70]}")
    
    print(f"\n📋 Sections around 100:")
    for rec in records:
        if 98 <= rec['section'] <= 105:
            title = rec['section_title'] if rec['section_title'] else "[NO TITLE]"
            print(f"  {rec['section']:3d}. {title[:70]}")
    
    # Check sections 1-100 have titles
    missing_titles = [r['section'] for r in records if r['section'] <= 100 and not r['section_title']]
    if missing_titles:
        print(f"\n⚠️ Sections 1-100 missing titles: {len(missing_titles)} sections")
        print(f"   Examples: {missing_titles[:10]}")
    else:
        print(f"\n✓ All sections 1-100 have titles!")
    
    # Find gaps
    sorted_sections = sorted(set(sections))
    gaps = []
    for i in range(len(sorted_sections) - 1):
        if sorted_sections[i+1] - sorted_sections[i] > 1:
            gaps.append((sorted_sections[i], sorted_sections[i+1]))
    
    if gaps:
        print(f"\n⚠️ Gaps found: {len(gaps)}")
        for start, end in gaps[:5]:
            print(f"  Gap: {start} → {end}")
    else:
        print(f"\n✓ No gaps in section numbering!")

Total lines extracted: 11691

✓ Total records: 1712
✓ Section range: 1 to 721

📋 First 25 sections with titles:
    1. Short title and commencement: (1) This Act may be cited as the
    1. Short title and commencement: (1) This Act may be cited as the
    2. Definitions: Unless the subject or the context otherwise requires, in
    3. Not to prejudice special legal provisions: Where a law in force
    4. Principles and provisions applicable generally: The principles
    5. Ignorance of law not to be excused: The ignorance of law shall
    6. Act against public interest not to be carried out: No one may
    7. Act contrary to law to be invalid: Any act contrary to law shall be
    8. A wrongdoer to bear liability for loss or damage: (1) A
    8. A wrongdoer to bear liability for loss or damage: (1) A
    8. A wrongdoer to bear liability for loss or damage: (1) A
    8. A wrongdoer to bear liability for loss or damage: (1) A
    9. Act causing nuisance to other not to be done: No person s

In [ ]:
#/kaggle/input/constitution444/Constitution-of-Nepal_2072_Eng_www.moljpa.gov_.npDate-72_11_16.pdf

In [16]:
import fitz
import re
import json

PDF_PATH = "/kaggle/input/constitution444/Constitution-of-Nepal_2072_Eng_www.moljpa.gov_.npDate-72_11_16.pdf"
OUTPUT = "bbbbbbbbbconstitution_clean.json"
LAW = "THE CONSTITUTION OF NEPAL"

doc = fitz.open(PDF_PATH)

# Extract text from all pages
all_text = []
for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()
    # Fix hyphenation
    text = re.sub(r'(\w)-\n(\w)', r'\1\2', text)
    all_text.append(text)

full_text = "\n".join(all_text)

# Split into lines and clean
lines = []
for line in full_text.split('\n'):
    line = line.strip()
    if line and not re.fullmatch(r'\d{1,2}', line):  # Skip standalone page numbers
        lines.append(line)

print(f"Total lines extracted: {len(lines)}")

# Patterns
part_re = re.compile(r'^Part[-–\s]*(\d+)', re.I)
chapter_re = re.compile(r'^Chapter[-–\s]*(\d+)', re.I)
# Match section number alone OR with title
section_num_only_re = re.compile(r'^(\d+)\.\s*$')  # Just "1." or "2."
section_with_title_re = re.compile(r'^(\d+)\.\s+(.+)$')  # "100. Title text"
subsection_re = re.compile(r'^\((\d+)\)\s*(.*)$')
clause_re = re.compile(r'^\(([a-z])\)\s*(.*)$')

# State
records = []
current_part = None
current_chapter = None
current_chapter_title = None
current_section = None
current_section_title = None
current_subsection = None
buffer = []

def flush():
    global buffer
    if current_section is not None and buffer:
        text = " ".join(buffer).strip()
        if text:
            records.append({
                "law": LAW,
                "part": current_part,
                "chapter": current_chapter,
                "chapter_title": current_chapter_title,
                "section": current_section,
                "section_title": current_section_title,
                "subsection": current_subsection,
                "text": text,
                "source": "civilcode-english.pdf"
            })
    buffer = []

i = 0
while i < len(lines):
    line = lines[i]
    
    # PART
    if part_re.match(line):
        flush()
        m = part_re.match(line)
        current_part = f"Part-{m.group(1)}"
        current_section = None
        current_subsection = None
        i += 1
        # Skip part title if exists
        if i < len(lines) and not section_num_only_re.match(lines[i]) and not section_with_title_re.match(lines[i]) and not chapter_re.match(lines[i]):
            i += 1
        continue
    
    # CHAPTER
    if chapter_re.match(line):
        flush()
        m = chapter_re.match(line)
        current_chapter = f"Chapter-{m.group(1)}"
        current_section = None
        current_subsection = None
        i += 1
        # Next line is chapter title
        if i < len(lines) and not section_num_only_re.match(lines[i]) and not section_with_title_re.match(lines[i]):
            current_chapter_title = lines[i]
            i += 1
        continue
    
    # SECTION WITH TITLE (like "100. Order to pay lump sum...")
    section_match = section_with_title_re.match(line)
    if section_match:
        section_num = int(section_match.group(1))
        if 1 <= section_num <= 999:
            flush()
            current_section = section_num
            current_section_title = section_match.group(2).strip().rstrip(':')
            current_subsection = None
            i += 1
            continue
    
    # SECTION NUMBER ONLY (like "1." or "2.")
    section_num_match = section_num_only_re.match(line)
    if section_num_match:
        section_num = int(section_num_match.group(1))
        if 1 <= section_num <= 999:
            flush()
            current_section = section_num
            current_subsection = None
            i += 1
            
            # CRITICAL: Next non-empty line is the section title
            while i < len(lines):
                next_line = lines[i].strip()
                
                # Skip empty lines
                if not next_line:
                    i += 1
                    continue
                
                # If next line is a subsection or clause, no title exists
                if subsection_re.match(next_line) or clause_re.match(next_line):
                    current_section_title = ""
                    break
                
                # If next line is another section/chapter/part, no title
                if (section_num_only_re.match(next_line) or 
                    section_with_title_re.match(next_line) or
                    chapter_re.match(next_line) or 
                    part_re.match(next_line)):
                    current_section_title = ""
                    break
                
                # Otherwise, this line is the section title
                current_section_title = next_line.rstrip(':').strip()
                i += 1
                break
            
            continue
    
    # SUBSECTION
    if subsection_re.match(line) and current_section is not None:
        flush()
        m = subsection_re.match(line)
        current_subsection = f"({m.group(1)})"
        rest = m.group(2).strip()
        if rest:
            buffer.append(rest)
        i += 1
        continue
    
    # CLAUSE or regular content
    if current_section is not None:
        buffer.append(line)
    
    i += 1

# Final flush
flush()

# Sort by section number
records.sort(key=lambda x: x['section'])

# Save
with open(OUTPUT, "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"\n{'='*60}")
print(f"✓ Total records: {len(records)}")

if records:
    sections = [r['section'] for r in records]
    print(f"✓ Section range: {min(sections)} to {max(sections)}")
    
    print(f"\n📋 First 25 sections with titles:")
    for rec in records[:25]:
        title = rec['section_title'] if rec['section_title'] else "[NO TITLE]"
        print(f"  {rec['section']:3d}. {title[:70]}")
    
    print(f"\n📋 Sections around 100:")
    for rec in records:
        if 98 <= rec['section'] <= 105:
            title = rec['section_title'] if rec['section_title'] else "[NO TITLE]"
            print(f"  {rec['section']:3d}. {title[:70]}")
    
    # Check sections 1-100 have titles
    missing_titles = [r['section'] for r in records if r['section'] <= 100 and not r['section_title']]
    if missing_titles:
        print(f"\n⚠️ Sections 1-100 missing titles: {len(missing_titles)} sections")
        print(f"   Examples: {missing_titles[:10]}")
    else:
        print(f"\n✓ All sections 1-100 have titles!")
    
    # Find gaps
    sorted_sections = sorted(set(sections))
    gaps = []
    for i in range(len(sorted_sections) - 1):
        if sorted_sections[i+1] - sorted_sections[i] > 1:
            gaps.append((sorted_sections[i], sorted_sections[i+1]))
    
    if gaps:
        print(f"\n⚠️ Gaps found: {len(gaps)}")
        for start, end in gaps[:5]:
            print(f"  Gap: {start} → {end}")
    else:
        print(f"\n✓ No gaps in section numbering!")

Total lines extracted: 7270

✓ Total records: 1187
✓ Section range: 1 to 308

📋 First 25 sections with titles:
    1. Constitution as the fundamental law: (1) This Constitution is the fund
    1. Constitution as the fundamental law: (1) This Constitution is the fund
    1. Nawalparasi (West
    1. Relating to defence and military
    1. Civil and criminal procedure, evidence and oaths (legal recognition,
    2. Sovereignty and state authority: The sovereignty and state authority o
    2. Operation of banks and financial institutions in accordance with the
    2. Supply, distribution, price control, quality and monitoring of essenti
    3. Nation: All the Nepalese people, with multiethnic, multilingual, multi
    3. Preventive detention for reasons connected with the security of the
    4. State of Nepal: (1) Nepal is an independent, indivisible, sovereign, s
    4. State of Nepal: (1) Nepal is an independent, indivisible, sovereign, s
    4. Central Police, Armed Police Force, national

In [ ]:
#/kaggle/input/procedure-criminal33/civil-procedure-code.pdf

In [ ]:
#/kaggle/input/procedure-criminal33/NP_Criminal Procedure Code_EN.pdf

In [17]:
import fitz
import re
import json

PDF_PATH = "/kaggle/input/procedure-criminal33/civil-procedure-code.pdf"
OUTPUT = "ccccccccccccc.json"
LAW = "the civil procedure-code"

doc = fitz.open(PDF_PATH)

# Extract text from all pages
all_text = []
for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()
    # Fix hyphenation
    text = re.sub(r'(\w)-\n(\w)', r'\1\2', text)
    all_text.append(text)

full_text = "\n".join(all_text)

# Split into lines and clean
lines = []
for line in full_text.split('\n'):
    line = line.strip()
    if line and not re.fullmatch(r'\d{1,2}', line):  # Skip standalone page numbers
        lines.append(line)

print(f"Total lines extracted: {len(lines)}")

# Patterns
part_re = re.compile(r'^Part[-–\s]*(\d+)', re.I)
chapter_re = re.compile(r'^Chapter[-–\s]*(\d+)', re.I)
# Match section number alone OR with title
section_num_only_re = re.compile(r'^(\d+)\.\s*$')  # Just "1." or "2."
section_with_title_re = re.compile(r'^(\d+)\.\s+(.+)$')  # "100. Title text"
subsection_re = re.compile(r'^\((\d+)\)\s*(.*)$')
clause_re = re.compile(r'^\(([a-z])\)\s*(.*)$')

# State
records = []
current_part = None
current_chapter = None
current_chapter_title = None
current_section = None
current_section_title = None
current_subsection = None
buffer = []

def flush():
    global buffer
    if current_section is not None and buffer:
        text = " ".join(buffer).strip()
        if text:
            records.append({
                "law": LAW,
                "part": current_part,
                "chapter": current_chapter,
                "chapter_title": current_chapter_title,
                "section": current_section,
                "section_title": current_section_title,
                "subsection": current_subsection,
                "text": text,
                "source": "civilcode-english.pdf"
            })
    buffer = []

i = 0
while i < len(lines):
    line = lines[i]
    
    # PART
    if part_re.match(line):
        flush()
        m = part_re.match(line)
        current_part = f"Part-{m.group(1)}"
        current_section = None
        current_subsection = None
        i += 1
        # Skip part title if exists
        if i < len(lines) and not section_num_only_re.match(lines[i]) and not section_with_title_re.match(lines[i]) and not chapter_re.match(lines[i]):
            i += 1
        continue
    
    # CHAPTER
    if chapter_re.match(line):
        flush()
        m = chapter_re.match(line)
        current_chapter = f"Chapter-{m.group(1)}"
        current_section = None
        current_subsection = None
        i += 1
        # Next line is chapter title
        if i < len(lines) and not section_num_only_re.match(lines[i]) and not section_with_title_re.match(lines[i]):
            current_chapter_title = lines[i]
            i += 1
        continue
    
    # SECTION WITH TITLE (like "100. Order to pay lump sum...")
    section_match = section_with_title_re.match(line)
    if section_match:
        section_num = int(section_match.group(1))
        if 1 <= section_num <= 999:
            flush()
            current_section = section_num
            current_section_title = section_match.group(2).strip().rstrip(':')
            current_subsection = None
            i += 1
            continue
    
    # SECTION NUMBER ONLY (like "1." or "2.")
    section_num_match = section_num_only_re.match(line)
    if section_num_match:
        section_num = int(section_num_match.group(1))
        if 1 <= section_num <= 999:
            flush()
            current_section = section_num
            current_subsection = None
            i += 1
            
            # CRITICAL: Next non-empty line is the section title
            while i < len(lines):
                next_line = lines[i].strip()
                
                # Skip empty lines
                if not next_line:
                    i += 1
                    continue
                
                # If next line is a subsection or clause, no title exists
                if subsection_re.match(next_line) or clause_re.match(next_line):
                    current_section_title = ""
                    break
                
                # If next line is another section/chapter/part, no title
                if (section_num_only_re.match(next_line) or 
                    section_with_title_re.match(next_line) or
                    chapter_re.match(next_line) or 
                    part_re.match(next_line)):
                    current_section_title = ""
                    break
                
                # Otherwise, this line is the section title
                current_section_title = next_line.rstrip(':').strip()
                i += 1
                break
            
            continue
    
    # SUBSECTION
    if subsection_re.match(line) and current_section is not None:
        flush()
        m = subsection_re.match(line)
        current_subsection = f"({m.group(1)})"
        rest = m.group(2).strip()
        if rest:
            buffer.append(rest)
        i += 1
        continue
    
    # CLAUSE or regular content
    if current_section is not None:
        buffer.append(line)
    
    i += 1

# Final flush
flush()

# Sort by section number
records.sort(key=lambda x: x['section'])

# Save
with open(OUTPUT, "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"\n{'='*60}")
print(f"✓ Total records: {len(records)}")

if records:
    sections = [r['section'] for r in records]
    print(f"✓ Section range: {min(sections)} to {max(sections)}")
    
    print(f"\n📋 First 25 sections with titles:")
    for rec in records[:25]:
        title = rec['section_title'] if rec['section_title'] else "[NO TITLE]"
        print(f"  {rec['section']:3d}. {title[:70]}")
    
    print(f"\n📋 Sections around 100:")
    for rec in records:
        if 98 <= rec['section'] <= 105:
            title = rec['section_title'] if rec['section_title'] else "[NO TITLE]"
            print(f"  {rec['section']:3d}. {title[:70]}")
    
    # Check sections 1-100 have titles
    missing_titles = [r['section'] for r in records if r['section'] <= 100 and not r['section_title']]
    if missing_titles:
        print(f"\n⚠️ Sections 1-100 missing titles: {len(missing_titles)} sections")
        print(f"   Examples: {missing_titles[:10]}")
    else:
        print(f"\n✓ All sections 1-100 have titles!")
    
    # Find gaps
    sorted_sections = sorted(set(sections))
    gaps = []
    for i in range(len(sorted_sections) - 1):
        if sorted_sections[i+1] - sorted_sections[i] > 1:
            gaps.append((sorted_sections[i], sorted_sections[i+1]))
    
    if gaps:
        print(f"\n⚠️ Gaps found: {len(gaps)}")
        for start, end in gaps[:5]:
            print(f"  Gap: {start} → {end}")
    else:
        print(f"\n✓ No gaps in section numbering!")

Total lines extracted: 6821

✓ Total records: 951
✓ Section range: 1 to 289

📋 First 25 sections with titles:
    1. Short title and commencement: (1) This Act may be cited as the
    1. Short title and commencement: (1) This Act may be cited as the
    1. I/we, the plaintiff submit this plaint as follows
    1. Mr./Ms. ……….., age of…..years, a resident of ………
    1. Mr./Ms. ……….., age of…..years, a resident of ………
    1. I/we have set out in the following clauses my/our actual details in
    1. Question: What is your and your father's, mother's name?
    1. Question: What is your and your father's, mother's name?
    1. Facts of the case in short: (to specify the facts of the case in short
    1. A case involving right, interest or concern of the Government of
    2. Definitions: Unless the subject or the context otherwise requires, in
    2. Definitions: Unless the subject or the context otherwise requires, in
    2. Definitions: Unless the subject or the context otherwise requires, 

In [18]:
import fitz
import re
import json

PDF_PATH = "/kaggle/input/procedure-criminal33/NP_Criminal Procedure Code_EN.pdf"
OUTPUT = "ddddddddd.json"
LAW = "the criminal procedure-code"

doc = fitz.open(PDF_PATH)

# Extract text from all pages
all_text = []
for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()
    # Fix hyphenation
    text = re.sub(r'(\w)-\n(\w)', r'\1\2', text)
    all_text.append(text)

full_text = "\n".join(all_text)

# Split into lines and clean
lines = []
for line in full_text.split('\n'):
    line = line.strip()
    if line and not re.fullmatch(r'\d{1,2}', line):  # Skip standalone page numbers
        lines.append(line)

print(f"Total lines extracted: {len(lines)}")

# Patterns
part_re = re.compile(r'^Part[-–\s]*(\d+)', re.I)
chapter_re = re.compile(r'^Chapter[-–\s]*(\d+)', re.I)
# Match section number alone OR with title
section_num_only_re = re.compile(r'^(\d+)\.\s*$')  # Just "1." or "2."
section_with_title_re = re.compile(r'^(\d+)\.\s+(.+)$')  # "100. Title text"
subsection_re = re.compile(r'^\((\d+)\)\s*(.*)$')
clause_re = re.compile(r'^\(([a-z])\)\s*(.*)$')

# State
records = []
current_part = None
current_chapter = None
current_chapter_title = None
current_section = None
current_section_title = None
current_subsection = None
buffer = []

def flush():
    global buffer
    if current_section is not None and buffer:
        text = " ".join(buffer).strip()
        if text:
            records.append({
                "law": LAW,
                "part": current_part,
                "chapter": current_chapter,
                "chapter_title": current_chapter_title,
                "section": current_section,
                "section_title": current_section_title,
                "subsection": current_subsection,
                "text": text,
                "source": "civilcode-english.pdf"
            })
    buffer = []

i = 0
while i < len(lines):
    line = lines[i]
    
    # PART
    if part_re.match(line):
        flush()
        m = part_re.match(line)
        current_part = f"Part-{m.group(1)}"
        current_section = None
        current_subsection = None
        i += 1
        # Skip part title if exists
        if i < len(lines) and not section_num_only_re.match(lines[i]) and not section_with_title_re.match(lines[i]) and not chapter_re.match(lines[i]):
            i += 1
        continue
    
    # CHAPTER
    if chapter_re.match(line):
        flush()
        m = chapter_re.match(line)
        current_chapter = f"Chapter-{m.group(1)}"
        current_section = None
        current_subsection = None
        i += 1
        # Next line is chapter title
        if i < len(lines) and not section_num_only_re.match(lines[i]) and not section_with_title_re.match(lines[i]):
            current_chapter_title = lines[i]
            i += 1
        continue
    
    # SECTION WITH TITLE (like "100. Order to pay lump sum...")
    section_match = section_with_title_re.match(line)
    if section_match:
        section_num = int(section_match.group(1))
        if 1 <= section_num <= 999:
            flush()
            current_section = section_num
            current_section_title = section_match.group(2).strip().rstrip(':')
            current_subsection = None
            i += 1
            continue
    
    # SECTION NUMBER ONLY (like "1." or "2.")
    section_num_match = section_num_only_re.match(line)
    if section_num_match:
        section_num = int(section_num_match.group(1))
        if 1 <= section_num <= 999:
            flush()
            current_section = section_num
            current_subsection = None
            i += 1
            
            # CRITICAL: Next non-empty line is the section title
            while i < len(lines):
                next_line = lines[i].strip()
                
                # Skip empty lines
                if not next_line:
                    i += 1
                    continue
                
                # If next line is a subsection or clause, no title exists
                if subsection_re.match(next_line) or clause_re.match(next_line):
                    current_section_title = ""
                    break
                
                # If next line is another section/chapter/part, no title
                if (section_num_only_re.match(next_line) or 
                    section_with_title_re.match(next_line) or
                    chapter_re.match(next_line) or 
                    part_re.match(next_line)):
                    current_section_title = ""
                    break
                
                # Otherwise, this line is the section title
                current_section_title = next_line.rstrip(':').strip()
                i += 1
                break
            
            continue
    
    # SUBSECTION
    if subsection_re.match(line) and current_section is not None:
        flush()
        m = subsection_re.match(line)
        current_subsection = f"({m.group(1)})"
        rest = m.group(2).strip()
        if rest:
            buffer.append(rest)
        i += 1
        continue
    
    # CLAUSE or regular content
    if current_section is not None:
        buffer.append(line)
    
    i += 1

# Final flush
flush()

# Sort by section number
records.sort(key=lambda x: x['section'])

# Save
with open(OUTPUT, "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"\n{'='*60}")
print(f"✓ Total records: {len(records)}")

if records:
    sections = [r['section'] for r in records]
    print(f"✓ Section range: {min(sections)} to {max(sections)}")
    
    print(f"\n📋 First 25 sections with titles:")
    for rec in records[:25]:
        title = rec['section_title'] if rec['section_title'] else "[NO TITLE]"
        print(f"  {rec['section']:3d}. {title[:70]}")
    
    print(f"\n📋 Sections around 100:")
    for rec in records:
        if 98 <= rec['section'] <= 105:
            title = rec['section_title'] if rec['section_title'] else "[NO TITLE]"
            print(f"  {rec['section']:3d}. {title[:70]}")
    
    # Check sections 1-100 have titles
    missing_titles = [r['section'] for r in records if r['section'] <= 100 and not r['section_title']]
    if missing_titles:
        print(f"\n⚠️ Sections 1-100 missing titles: {len(missing_titles)} sections")
        print(f"   Examples: {missing_titles[:10]}")
    else:
        print(f"\n✓ All sections 1-100 have titles!")
    
    # Find gaps
    sorted_sections = sorted(set(sections))
    gaps = []
    for i in range(len(sorted_sections) - 1):
        if sorted_sections[i+1] - sorted_sections[i] > 1:
            gaps.append((sorted_sections[i], sorted_sections[i+1]))
    
    if gaps:
        print(f"\n⚠️ Gaps found: {len(gaps)}")
        for start, end in gaps[:5]:
            print(f"  Gap: {start} → {end}")
    else:
        print(f"\n✓ No gaps in section numbering!")

Total lines extracted: 7645

✓ Total records: 947
✓ Section range: 1 to 198

📋 First 25 sections with titles:
    1. Short title and commencement: (1) This Act may be cited as the
    1. Short title and commencement: (1) This Act may be cited as the
    1. Offences under the following Chapters or under the following
    1. Offences under the following Sections of the following Chapter of
    1. Offences under the following Chapters or under the following
    1. Name, surname and address of the person making the first
    1. Short description of the first information report or notice
    1. Name, surname, description of the deceased
    1. Opinion or cause of death
    1. Date and time of receipt of information or complaint of
    2. Definitions: Unless the subject or the context otherwise requires, in
    2. Definitions: Unless the subject or the context otherwise requires, in
    2. Definitions: Unless the subject or the context otherwise requires, in
    2. Definitions: Unless the su

In [19]:
import fitz
import re
import json

PDF_PATH = "/kaggle/input/constitution444/Constitution-of-Nepal_2072_Eng_www.moljpa.gov_.npDate-72_11_16.pdf"
OUTPUT = "eeeeeeeeee.json"
LAW = "the constitution of nepal "

doc = fitz.open(PDF_PATH)

# Extract text from all pages
all_text = []
for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()
    # Fix hyphenation
    text = re.sub(r'(\w)-\n(\w)', r'\1\2', text)
    all_text.append(text)

full_text = "\n".join(all_text)

# Split into lines and clean
lines = []
for line in full_text.split('\n'):
    line = line.strip()
    if line and not re.fullmatch(r'\d{1,2}', line):  # Skip standalone page numbers
        lines.append(line)

print(f"Total lines extracted: {len(lines)}")

# Patterns
part_re = re.compile(r'^Part[-–\s]*(\d+)', re.I)
chapter_re = re.compile(r'^Chapter[-–\s]*(\d+)', re.I)
section_num_only_re = re.compile(r'^(\d+)\.\s*$')  # Just "1." or "2."
section_with_title_re = re.compile(r'^(\d+)\.\s+(.+)$')  # "100. Title text"
subsection_re = re.compile(r'^\((\d+)\)\s*(.*)$')
clause_re = re.compile(r'^\(([a-z])\)\s*(.*)$')

# State
records = []
current_part = None
current_chapter = None
current_chapter_title = None
current_section = None
current_section_title = None
buffer = []  # Collects ALL text for the current section (including subsections)

def flush():
    """Save the complete section with all its subsections as one record"""
    global buffer
    if current_section is not None and buffer:
        text = " ".join(buffer).strip()
        if text:
            records.append({
                "law": LAW,
                "part": current_part,
                "chapter": current_chapter,
                "chapter_title": current_chapter_title,
                "section": current_section,
                "section_title": current_section_title,
                "text": text,  # Contains all subsections
                "source": "civil-procedure-code.pdf"
            })
    buffer = []

i = 0
while i < len(lines):
    line = lines[i]
    
    # PART
    if part_re.match(line):
        flush()  # Save previous section
        m = part_re.match(line)
        current_part = f"Part-{m.group(1)}"
        current_section = None
        i += 1
        # Skip part title if exists
        if i < len(lines) and not section_num_only_re.match(lines[i]) and not section_with_title_re.match(lines[i]) and not chapter_re.match(lines[i]):
            i += 1
        continue
    
    # CHAPTER
    if chapter_re.match(line):
        flush()  # Save previous section
        m = chapter_re.match(line)
        current_chapter = f"Chapter-{m.group(1)}"
        current_section = None
        i += 1
        # Next line is chapter title
        if i < len(lines) and not section_num_only_re.match(lines[i]) and not section_with_title_re.match(lines[i]):
            current_chapter_title = lines[i]
            i += 1
        continue
    
    # SECTION WITH TITLE (like "100. Order to pay lump sum...")
    section_match = section_with_title_re.match(line)
    if section_match:
        section_num = int(section_match.group(1))
        if 1 <= section_num <= 999:
            flush()  # Save previous section
            current_section = section_num
            current_section_title = section_match.group(2).strip().rstrip(':')
            buffer = []  # Start fresh for new section
            i += 1
            continue
    
    # SECTION NUMBER ONLY (like "1." or "2.")
    section_num_match = section_num_only_re.match(line)
    if section_num_match:
        section_num = int(section_num_match.group(1))
        if 1 <= section_num <= 999:
            flush()  # Save previous section
            current_section = section_num
            buffer = []  # Start fresh for new section
            i += 1
            
            # CRITICAL: Next non-empty line is the section title
            while i < len(lines):
                next_line = lines[i].strip()
                
                # Skip empty lines
                if not next_line:
                    i += 1
                    continue
                
                # If next line is a subsection or clause, no title exists
                if subsection_re.match(next_line) or clause_re.match(next_line):
                    current_section_title = ""
                    break
                
                # If next line is another section/chapter/part, no title
                if (section_num_only_re.match(next_line) or 
                    section_with_title_re.match(next_line) or
                    chapter_re.match(next_line) or 
                    part_re.match(next_line)):
                    current_section_title = ""
                    break
                
                # Otherwise, this line is the section title
                current_section_title = next_line.rstrip(':').strip()
                i += 1
                break
            
            continue
    
    # SUBSECTION - NO flush(), just add to buffer
    if subsection_re.match(line) and current_section is not None:
        buffer.append(line)  # Keep subsection marker with text
        i += 1
        continue
    
    # CLAUSE or regular content - add to buffer
    if current_section is not None:
        buffer.append(line)
    
    i += 1

# Final flush
flush()

# Sort by section number
records.sort(key=lambda x: x['section'])

# Save
with open(OUTPUT, "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"\n{'='*60}")
print(f"✓ Total records: {len(records)}")

if records:
    sections = [r['section'] for r in records]
    print(f"✓ Section range: {min(sections)} to {max(sections)}")
    print(f"✓ Unique sections: {len(set(sections))}")
    
    print(f"\n📋 First 15 sections:")
    for rec in records[:15]:
        title = rec['section_title'] if rec['section_title'] else "[NO TITLE]"
        text_preview = rec['text'][:80] + "..." if len(rec['text']) > 80 else rec['text']
        print(f"  {rec['section']:3d}. {title}")
        print(f"       Text: {text_preview}")
    
    print(f"\n📋 Sample section with subsections (Section 10):")
    for rec in records:
        if rec['section'] == 10:
            print(f"  Section: {rec['section']}")
            print(f"  Title: {rec['section_title']}")
            print(f"  Full text ({len(rec['text'])} chars):")
            print(f"  {rec['text'][:300]}...")
            break
    
    # Check for duplicates
    from collections import Counter
    section_counts = Counter([r['section'] for r in records])
    duplicates = {sec: count for sec, count in section_counts.items() if count > 1}
    
    if duplicates:
        print(f"\n⚠️ Duplicate sections found: {len(duplicates)}")
        for sec, count in sorted(duplicates.items())[:10]:
            print(f"  Section {sec}: appears {count} times")
    else:
        print(f"\n✓ No duplicate sections!")
    
    # Check sections 1-100 have titles
    missing_titles = [r['section'] for r in records if r['section'] <= 100 and not r['section_title']]
    if missing_titles:
        print(f"\n⚠️ Sections 1-100 missing titles: {len(missing_titles)} sections")
        print(f"   Examples: {missing_titles[:10]}")
    else:
        print(f"\n✓ All sections 1-100 have titles!")
    
    # Find gaps
    sorted_sections = sorted(set(sections))
    gaps = []
    for i in range(len(sorted_sections) - 1):
        if sorted_sections[i+1] - sorted_sections[i] > 1:
            gaps.append((sorted_sections[i], sorted_sections[i+1]))
    
    if gaps:
        print(f"\n⚠️ Gaps found: {len(gaps)}")
        for start, end in gaps[:5]:
            print(f"  Gap: {start} → {end}")
    else:
        print(f"\n✓ No gaps in section numbering!")

Total lines extracted: 7270

✓ Total records: 361
✓ Section range: 1 to 308
✓ Unique sections: 307

📋 First 15 sections:
    1. Constitution as the fundamental law: (1) This Constitution is the fundamental
       Text: law of Nepal. Any law inconsistent with this Constitution shall, to the extent o...
    1. Nawalparasi (West
       Text: of Bardaghat Susta
    1. Relating to defence and military
       Text: (a) Protection of national unity and territorial integrity (b) Relating to natio...
    1. Civil and criminal procedure, evidence and oaths (legal recognition,
       Text: public acts and records, and judicial proceedings)
    2. Sovereignty and state authority: The sovereignty and state authority of Nepal
       Text: shall be vested in the Nepalese people. It shall be exercised in accordance with...
    2. Operation of banks and financial institutions in accordance with the
       Text: policies of Nepal Rastra Bank, cooperative institutions, foreign grants and assi...
    2. S

In [21]:
import fitz
import re
import json

PDF_PATH = "/kaggle/input/constitution444/Constitution-of-Nepal_2072_Eng_www.moljpa.gov_.npDate-72_11_16.pdf"
OUTPUT = "fffffff.json"
LAW = "the constitution of nepal "

doc = fitz.open(PDF_PATH)

# Extract text from all pages
all_text = []
for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()
    text = re.sub(r'(\w)-\n(\w)', r'\1\2', text)  # Fix hyphenation
    all_text.append(text)

full_text = "\n".join(all_text)
lines = [l.strip() for l in full_text.split('\n') if l.strip()]

# Remove standalone page numbers (1-3 digits only)
lines = [l for l in lines if not re.fullmatch(r'\d{1,3}', l)]

print(f"Total lines: {len(lines)}")

# STRICT patterns
part_re = re.compile(r'^Part[-–\s]*(\d+)\s*$', re.I)  # Must be on own line
chapter_re = re.compile(r'^Chapter[-–\s]*(\d+)\s*$', re.I)  # Must be on own line
# Section must start at line beginning with number, period, and space
section_num_only_re = re.compile(r'^(\d+)\.\s*$')
section_with_title_re = re.compile(r'^(\d+)\.\s+([A-Z][^:]*(?::|$))', re.UNICODE)
subsection_re = re.compile(r'^\((\d+)\)')
clause_re = re.compile(r'^\(([a-z])\)')

# State
sections_data = {}  # {section_num: {metadata + text}}
current_part = None
current_chapter = None
current_chapter_title = None
current_section = None
current_section_title = None
section_buffer = []

def save_section():
    """Save current section to dictionary"""
    global section_buffer
    if current_section and section_buffer:
        # Only save if this section doesn't exist yet, or merge text
        text = " ".join(section_buffer).strip()
        if current_section not in sections_data:
            sections_data[current_section] = {
                "law": LAW,
                "part": current_part,
                "chapter": current_chapter,
                "chapter_title": current_chapter_title,
                "section": current_section,
                "section_title": current_section_title,
                "text": text,
                "source": "https://www.wipo.int/wipolex/edocs/lexdocs/laws/en/np/np029en.pdf?utm_source=chatgpt.com"
            }
        else:
            # Append to existing text (in case section continues)
            sections_data[current_section]["text"] += " " + text
    section_buffer = []

# Track if we're in actual legal content
in_legal_content = False

i = 0
while i < len(lines):
    line = lines[i]
    
    # PART
    part_match = part_re.match(line)
    if part_match:
        save_section()
        current_part = f"Part-{part_match.group(1)}"
        current_section = None
        in_legal_content = True
        print(f"Found {current_part}")
        i += 1
        # Skip part title
        if i < len(lines) and not (section_num_only_re.match(lines[i]) or 
                                   section_with_title_re.match(lines[i]) or 
                                   chapter_re.match(lines[i])):
            i += 1
        continue
    
    # CHAPTER
    chapter_match = chapter_re.match(line)
    if chapter_match:
        save_section()
        current_chapter = f"Chapter-{chapter_match.group(1)}"
        current_section = None
        i += 1
        # Next line is chapter title
        if i < len(lines) and not (section_num_only_re.match(lines[i]) or 
                                   section_with_title_re.match(lines[i])):
            current_chapter_title = lines[i]
            print(f"  {current_chapter}: {current_chapter_title}")
            i += 1
        continue
    
    # Skip lines until we're in legal content
    if not in_legal_content:
        i += 1
        continue
    
    # SECTION WITH TITLE (e.g., "1. Short title:")
    section_with_title_match = section_with_title_re.match(line)
    if section_with_title_match:
        section_num = int(section_with_title_match.group(1))
        # Must be reasonable section number
        if 1 <= section_num <= 1000:
            save_section()
            current_section = section_num
            current_section_title = section_with_title_match.group(2).strip().rstrip(':')
            section_buffer = []
            if section_num <= 20:
                print(f"    Section {section_num}: {current_section_title}")
            i += 1
            continue
    
    # SECTION NUMBER ONLY (e.g., "1.")
    section_num_only_match = section_num_only_re.match(line)
    if section_num_only_match:
        section_num = int(section_num_only_match.group(1))
        if 1 <= section_num <= 1000:
            save_section()
            current_section = section_num
            section_buffer = []
            i += 1
            
            # Look for title on next line
            if i < len(lines):
                next_line = lines[i].strip()
                # Check if next line is title (starts with capital, not subsection/clause)
                if (next_line and 
                    not subsection_re.match(next_line) and 
                    not clause_re.match(next_line) and
                    not section_num_only_re.match(next_line) and
                    not section_with_title_re.match(next_line) and
                    next_line[0].isupper()):
                    current_section_title = next_line.rstrip(':')
                    if section_num <= 20:
                        print(f"    Section {section_num}: {current_section_title}")
                    i += 1
                else:
                    current_section_title = ""
            continue
    
    # Content for current section (including subsections)
    if current_section is not None:
        section_buffer.append(line)
    
    i += 1

# Save last section
save_section()

# Convert to list and sort
records = sorted(sections_data.values(), key=lambda x: x['section'])

# Save JSON
with open(OUTPUT, "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"\n{'='*70}")
print(f"✓ Total sections: {len(records)}")
if records:
    print(f"✓ Section range: {records[0]['section']} to {records[-1]['section']}")
    
    print(f"\n📋 First 10 sections:")
    for rec in records[:10]:
        title = rec['section_title'] or "[NO TITLE]"
        text_len = len(rec['text'])
        print(f"  Section {rec['section']:3d}: {title[:60]}")
        print(f"              ({text_len} chars)")
    
    print(f"\n📋 Sample section detail (Section 1):")
    sec1 = next((r for r in records if r['section'] == 1), None)
    if sec1:
        print(f"  Title: {sec1['section_title']}")
        print(f"  Part: {sec1['part']}")
        print(f"  Chapter: {sec1['chapter']}")
        print(f"  Text preview: {sec1['text'][:200]}...")
    
    # Check for gaps
    section_nums = sorted([r['section'] for r in records])
    gaps = []
    for i in range(len(section_nums) - 1):
        if section_nums[i+1] - section_nums[i] > 1:
            gaps.append((section_nums[i], section_nums[i+1]))
    
    if gaps:
        print(f"\n⚠️ Gaps in numbering: {len(gaps)}")
        for start, end in gaps[:5]:
            print(f"  {start} → {end}")
    else:
        print(f"\n✓ No gaps!")
    
    # Check duplicates
    if len(records) != len(section_nums):
        print(f"\n⚠️ Warning: {len(records)} records but {len(section_nums)} unique sections")
    else:
        print(f"\n✓ All sections unique!")

Total lines: 7129
Found Part-1
Found Part-2
Found Part-3
Found Part-4
Found Part-5
Found Part-6
Found Part-7
Found Part-8
Found Part-9
Found Part-10
Found Part-11
Found Part-12
Found Part-13
Found Part-14
Found Part-15
Found Part-16
Found Part-17
Found Part-18
Found Part-19
Found Part-20
Found Part-21
Found Part-22
Found Part-23
Found Part-24
Found Part-25
Found Part-26
Found Part-27
Found Part-28
Found Part-29
Found Part-30
Found Part-31
Found Part-32
Found Part-33
Found Part-34
Found Part-35
Found Part-1
    Section 1: Constitution as the fundamental law: (1) This Constitution is the fundamental
    Section 2: Sovereignty and state authority: The sovereignty and state authority of Nepal
    Section 3: Nation: All the Nepalese people, with multiethnic, multilingual, multi-religious,
    Section 4: State of Nepal: (1) Nepal is an independent, indivisible, sovereign, secular,
    Section 5: National interest: (1) Safeguarding of the freedom, sovereignty, territorial
    Section 6: Langu